In [2]:
import sqlite3
import pandas as pd
import numpy as np

In [1]:
import utils

In [4]:
import utils
from utils import TimedSQLiteConnection, get_query, get_query_one, get_query_dict, \
get_query_course_dict, get_query_df

In [11]:
conn = TimedSQLiteConnection('UMGC.db')

In [8]:
student_info = utils.initialize_student_info()
student_info

{'user_id': None,
 'name': None,
 'financial_aid': None,
 'resident_status': None,
 'student_profile': None,
 'transfer_credits': None,
 'app_stage': None,
 'app_stage_id': None,
 'first_term': None,
 'program_id': None,
 'degree_program': None,
 'menu': {'degree': None, 'area_of_study': None, 'program': None}}

In [9]:
student_data = utils.initialize_student_data()
student_data

{'user_id': None, 'required': None, 'periods': None, 'schedule': None}

In [10]:
user_id = 3
student_info['user_id'] = user_id
student_data['user_id'] = user_id

In [12]:
query = 'SELECT * FROM student_info_view WHERE user_id = ?'
timedConnection = conn
row = await get_query_one(timedConnection, query, params=(user_id,))


In [15]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
              'transfer_credits', 'program_id']
student_info.update({name: row[name] for name in attributes})
student_info

{'user_id': 3,
 'name': None,
 'financial_aid': 1,
 'resident_status': 'In-State',
 'student_profile': 'Full-time',
 'transfer_credits': 0,
 'app_stage': 'schedule created',
 'app_stage_id': 4,
 'first_term': None,
 'program_id': 5,
 'degree_program': None,
 'menu': {'degree': None, 'area_of_study': None, 'program': None}}

In [16]:
row = await utils.get_program_title(timedConnection, student_info['program_id'])
if row:
    student_info['degree_program'] = row['title']
    student_info['degree_id'] = row['id']
student_info

{'user_id': 3,
 'name': None,
 'financial_aid': 1,
 'resident_status': 'In-State',
 'student_profile': 'Full-time',
 'transfer_credits': 0,
 'app_stage': 'schedule created',
 'app_stage_id': 4,
 'first_term': None,
 'program_id': 5,
 'degree_program': 'BS in Business Administration',
 'menu': {'degree': None, 'area_of_study': None, 'program': None},
 'degree_id': 3}

In [17]:
student_data

{'user_id': 3, 'required': None, 'periods': None, 'schedule': None}

In [18]:
query = 'SELECT * FROM student_progress_d3_view WHERE user_id = ?'
# note: 'course' is named 'name' in student_progress_d3_view 
df = await get_query_df(conn, query, params=(student_info['user_id'],))
student_data['schedule'] = df

In [19]:
df

,id,user_id,seq,name,title,credits,course_type,type,completed,term,session,locked,prerequisites,pre,pre_credits,substitutions,description
0,1,3,1,PACE 111B,Program and Career Exploration in Business,3,general,general,0,1,1,1,,,,"PACE 111B, PACE 111C, PACE 111M, PACE 111P, PA...",(Fulfills the general education requirement in...
1,2,3,2,LIBS 150,Introduction to Research,1,general,general,0,1,1,1,,,,"COMP 111, LIBS 100, or LIBS 150",An introduction to the research process and me...
2,3,3,3,WRTG 111,Academic Writing I,3,general,general,0,1,3,0,,,,"WRTG 100A, WRTG 111, or WRTG 111X",(The first course in the two-course series WRT...
3,4,3,4,NUTR 100,Elements of Nutrition,3,general,general,0,1,3,0,,,,NUTR 100 or NUTR 200,A study of the scientific and quantitative fou...
4,5,3,5,WRTG 112,Academic Writing II,3,general,general,0,2,1,0,,,,"ENGL 101, ENGL 101X, WRTG 101, WRTG 101S, WRTG...",(The second course in the two-course series WR...
5,6,3,6,BMGT 110,Introduction to Business and Management,3,major,major,0,2,2,0,,,,,(For students with little or no business backg...
6,7,3,7,SPCH 100,Foundations of Oral Communication,3,general,general,0,3,1,0,,,,"SPCH 100, SPCH 100X, SPCH 101, SPCH 107, or SP...","For online sections, access to a broadband int..."
7,8,3,8,ELECTIVE,,3,elective,elective,0,3,1,0,,,,,
8,9,3,9,STAT 200,Introduction to Statistics,3,"required,general",required,0,3,3,0,,,,"BEHS 202, BEHS 302, BMGT 230, ECON 321, GNST 2...",An introduction to statistics. The objective i...
9,10,3,10,IFSM 300,Information Systems in Organizations,3,"required,general",required,0,3,3,0,,,,,An overview of information systems and how the...


In [20]:
html_template = utils.create_html_template(df, 'SPRING 2024')
html_template

'\n<!DOCTYPE html>\n<html>\n<head>\n  <style>\n    body {\n      text-align: center;\n    }   \n    svg {\n      margin-top: 12px;\n      border: 1px solid #aaa;\n    }\n  </style>\n  <script src=\'https://d3js.org/d3.v5.js\'></script>\n</head>\n<body>\n  <script type="module">\n    \n    //const screenWidth = 1200;\n    const screenWidth = 800;\n    const boxHeight = 40;\n    const textOffsetX = 20;\n    const textOffsetY = 25;\n\n    function drawRect(x, y, width, printname, color, textcolor, offset, fontsize, description) {\n        var g = zoomable.append("g");\n        g.append("rect")\n            .attr("x", x)\n            .attr("y", y)\n            .attr("width", width)\n            .attr("height", boxHeight)\n            .style("fill", color)\n            .classed("movable", true); \n        g.append("text")\n            .attr("x", x + offset*textOffsetX)\n            .attr("y", y + textOffsetY)\n            .text(printname)\n            .attr("fill", textcolor)\n            .

In [25]:
df_input = df.copy()
df_input.rename(columns={'term': 'period'}, inplace=True)
df_display, headers_display = utils.prepare_d3_data(df_input)

In [26]:
df_display

,x,y,width,printname,color,textcolor,offset,fontsize,period,session,type,name,credits,description
0,10.0,70,120,PACE 111B (3),#3b8132,white,1,12px,1,1,general,PACE 111B,3,
1,10.0,115,120,LIBS 150 (1),#3b8132,white,1,12px,1,1,general,LIBS 150,1,
2,150.0,160,120,WRTG 111 (3),#3b8132,white,1,12px,1,3,general,WRTG 111,3,
3,150.0,205,120,NUTR 100 (3),#3b8132,white,1,12px,1,3,general,NUTR 100,3,
4,310.0,70,120,WRTG 112 (3),#3b8132,white,1,12px,2,1,general,WRTG 112,3,
5,380.0,115,120,BMGT 110 (3),#135f96,white,1,12px,2,2,major,BMGT 110,3,
6,540.0,70,120,SPCH 100 (3),#3b8132,white,1,12px,3,1,general,SPCH 100,3,
7,540.0,115,120,ELECTIVE (3),#fdbf38,black,1,12px,3,1,elective,ELECTIVE,3,
8,680.0,160,120,STAT 200 (3),#a30606,white,1,12px,3,3,required,STAT 200,3,
9,680.0,205,120,IFSM 300 (3),#a30606,white,1,12px,3,3,required,IFSM 300,3,


In [27]:
headers_display

,x,y,width,printname,color,textcolor,offset,fontsize,period,name,credits,description
0,10.0,10,260,SPRING 2024 (10),lightgray,black,3,14px,1,SPRING 2024,10,
1,310.0,10,190,SUMMER 2024 (6),lightgray,black,2,14px,2,SUMMER 2024,6,
2,540.0,10,260,FALL 2024 (12),lightgray,black,3,14px,3,FALL 2024,12,
3,840.0,10,260,WINTER 2025 (12),lightgray,black,3,14px,4,WINTER 2025,12,
4,1140.0,10,260,SPRING 2025 (13),lightgray,black,3,14px,5,SPRING 2025,13,
5,1440.0,10,190,SUMMER 2025 (6),lightgray,black,2,14px,6,SUMMER 2025,6,
6,1670.0,10,260,FALL 2025 (12),lightgray,black,3,14px,7,FALL 2025,12,
7,1970.0,10,260,WINTER 2026 (12),lightgray,black,3,14px,8,WINTER 2026,12,
8,2270.0,10,260,SPRING 2026 (12),lightgray,black,3,14px,9,SPRING 2026,12,
9,2570.0,10,190,SUMMER 2026 (6),lightgray,black,2,14px,10,SUMMER 2026,6,


In [32]:
periods = utils.generate_periods(summer=True)
periods

,id,term,session,year,max_courses,max_credits,previous
0,1,SPRING,1,2024,3,18,1
1,2,SPRING,2,2024,0,0,2
2,3,SPRING,3,2024,3,18,2
3,4,SUMMER,1,2024,3,12,1
4,5,SUMMER,2,2024,3,12,2
...,...,...,...,...,...,...,...
80,81,SUMMER,1,2031,3,12,1
81,82,SUMMER,2,2031,3,12,2
82,83,FALL,1,2031,3,18,1
83,84,FALL,2,2031,0,0,2


In [37]:
def parse_prerequisites(prereq_string):
    '''
    Parse the prerequisites string and return a list of prerequisite groups.
    Each group contains lists of courses that satisfy the 'or' and 'and' patterns.
    '''
    import re
    prereq_string = prereq_string.strip()
    prereq_string = re.sub(r'\s+', ' ', prereq_string)
    prereq_string = re.sub(r'\(', '', prereq_string)
    prereq_string = re.sub(r'\)', '', prereq_string)
    
    and_groups = prereq_string.split('&')
    prereq_groups = [group.split('|') for group in and_groups]
    
    # Strip whitespace from each course in the groups
    prereq_groups = [[course.strip() for course in group] for group in prereq_groups]
    
    return prereq_groups


In [38]:
parse_prerequisites('(CMIT 202 | CMIT 265)')

[['CMIT 202', 'CMIT 265']]

In [40]:
parse_prerequisites('(STAT 200 & (MATH 115 | MATH 108))')

[['STAT 200'], ['MATH 115', 'MATH 108']]

    id    term  session  year  max_courses  max_credits  previous
0    1  SPRING        1  2024            3           18         1
1    2  SPRING        2  2024            0            0         2
2    3  SPRING        3  2024            3           18         2
3    4  SUMMER        1  2024            3           12         1
4    5  SUMMER        2  2024            3           12         2
..  ..     ...      ...   ...          ...          ...       ...
80  81  SUMMER        1  2031            3           12         1
81  82  SUMMER        2  2031            3           12         2
82  83    FALL        1  2031            3           18         1
83  84    FALL        2  2031            0            0         2
84  85    FALL        3  2031            3           18         2

[85 rows x 7 columns]


In [3]:
dir(keycloak)

['AsyncAuthenticationMixin',
 'AsyncAuthorizationMixin',
 'AsyncClient',
 'AsyncResourceMixin',
 'AsyncTokenMixin',
 'AuthenticationMixin',
 'AuthorizationMixin',
 'Client',
 'ResourceMixin',
 'Singleton',
 'TokenMixin',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'config',
 'constants',
 'core',
 'utils']

In [2]:
import keycloak

In [ ]:
#from keycloak import KeycloakOpenID
#from keycloak.exceptions import KeycloakAuthenticationError

ImportError: cannot import name 'KeycloakOpenID' from 'keycloak' (/Users/david/AspenCove/catalog/wave/.venv/lib/python3.10/site-packages/keycloak/__init__.py)

In [ ]:
keycloak

In [ ]:
# Configuration
keycloak_server_url = "http://localhost:8080/auth/"
keycloak_realm_name = "your-realm"
keycloak_client_id = "your-client-id"
keycloak_client_secret = "your-client-secret"  # If using a confidential client
keycloak_redirect_uri = "http://localhost:8888/callback"
keycloak_username = "sample-user"
keycloak_password = "sample-password"

# Initialize Keycloak client
keycloak_openid = KeycloakOpenID(server_url=keycloak_server_url,
                                 client_id=keycloak_client_id,
                                 realm_name=keycloak_realm_name,
                                 client_secret_key=keycloak_client_secret)

# Authenticate and obtain access token
try:
    token = keycloak_openid.token(keycloak_username, keycloak_password)
    access_token = token['access_token']
    print("Access token:", access_token)
except KeycloakAuthenticationError as e:
    print(f"Failed to authenticate: {e}")

# Fetch user information
if access_token:
    userinfo = keycloak_openid.userinfo(access_token)
    print("User information:", userinfo)

In [42]:
'  My Name'.strip()

'My Name'

In [46]:
timedConnection = conn
program_id = 5
user_id = 3
student_data = {}

In [47]:
query = 'SELECT * FROM student_progress_d3_view WHERE user_id = ?'
df = await get_query_df(timedConnection, query, params=(user_id,))
student_data['schedule'] = df
df

,id,user_id,seq,name,title,credits,course_type,type,completed,term,session,locked,prerequisites,pre,pre_credits,substitutions,description
0,1,3,1,PACE 111B,Program and Career Exploration in Business,3,general,general,0,1,1,1,,,,"PACE 111B, PACE 111C, PACE 111M, PACE 111P, PA...",(Fulfills the general education requirement in...
1,2,3,2,LIBS 150,Introduction to Research,1,general,general,0,1,1,1,,,,"COMP 111, LIBS 100, or LIBS 150",An introduction to the research process and me...
2,3,3,3,WRTG 111,Academic Writing I,3,general,general,0,1,3,0,,,,"WRTG 100A, WRTG 111, or WRTG 111X",(The first course in the two-course series WRT...
3,4,3,4,NUTR 100,Elements of Nutrition,3,general,general,0,1,3,0,,,,NUTR 100 or NUTR 200,A study of the scientific and quantitative fou...
4,5,3,5,WRTG 112,Academic Writing II,3,general,general,0,2,1,0,,,,"ENGL 101, ENGL 101X, WRTG 101, WRTG 101S, WRTG...",(The second course in the two-course series WR...
5,6,3,6,BMGT 110,Introduction to Business and Management,3,major,major,0,2,2,0,,,,,(For students with little or no business backg...
6,7,3,7,SPCH 100,Foundations of Oral Communication,3,general,general,0,3,1,0,,,,"SPCH 100, SPCH 100X, SPCH 101, SPCH 107, or SP...","For online sections, access to a broadband int..."
7,8,3,8,ELECTIVE,,3,elective,elective,0,3,1,0,,,,,
8,9,3,9,STAT 200,Introduction to Statistics,3,"required,general",required,0,3,3,0,,,,"BEHS 202, BEHS 302, BMGT 230, ECON 321, GNST 2...",An introduction to statistics. The objective i...
9,10,3,10,IFSM 300,Information Systems in Organizations,3,"required,general",required,0,3,3,0,,,,,An overview of information systems and how the...


In [49]:
query = '''
        SELECT 
            id,
            course, 
            course_type as type,
            title,
            credits,
            pre,
            pre_credits,
            substitutions,
            description
        FROM program_requirements_view
        WHERE program_id = ?
'''
df = await get_query_df(timedConnection, query, params=(program_id,))
student_data['required'] = df
df

,id,course,type,title,credits,pre,pre_credits,substitutions,description
0,22,BMGT 110,Major,Introduction to Business and Management,3,,,,(For students with little or no business backg...
1,23,ACCT 220,Major,Principles of Accounting I,3,,,,An introduction to the basic theory and techni...
2,24,ACCT 221,Major,Principles of Accounting II,3,ACCT 220,,,Prerequisite: ACCT 220. Further study of conte...
3,25,BMGT 364,Major,Management and Organization Theory,3,,,,Recommended: BMGT 110. An examination of the f...
4,26,BMGT 365,Major,Organizational Leadership,3,(BMGT 110 | BMGT 364),,,Prerequisite: BMGT 110 or BMGT 364. An explora...
5,27,MRKT 310,Major,Marketing Principles,3,,,,A foundational study of the marketing principl...
6,28,BMGT 380,Major,Business Law I,3,,,,(Strongly recommended for students seeking car...
7,29,HRMN 300,Major,Human Resource Management,3,,,,A basic study of the strategic role of human r...
8,30,FINC 330,Major,Business Finance,3,(ACCT 221 & STAT 200),,,Prerequisites: ACCT 221 and STAT 200. An overv...
9,31,BMGT 496,Major,Business Ethics,3,,,,A study of the relationship of business ethics...


In [50]:
student_data

{'schedule':     id  user_id  seq       name                                       title  \
 0    1        3    1  PACE 111B  Program and Career Exploration in Business   
 1    2        3    2   LIBS 150                    Introduction to Research   
 2    3        3    3   WRTG 111                          Academic Writing I   
 3    4        3    4   NUTR 100                       Elements of Nutrition   
 4    5        3    5   WRTG 112                         Academic Writing II   
 5    6        3    6   BMGT 110     Introduction to Business and Management   
 6    7        3    7   SPCH 100           Foundations of Oral Communication   
 7    8        3    8   ELECTIVE                                               
 8    9        3    9   STAT 200                  Introduction to Statistics   
 9   10        3   10   IFSM 300        Information Systems in Organizations   
 10  11        3   11   ACCT 220                  Principles of Accounting I   
 11  12        3   12   ELEC

In [51]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
        'transfer_credits', 'program_id']

In [52]:
attributes

['resident_status',
 'app_stage_id',
 'app_stage',
 'student_profile',
 'financial_aid',
 'transfer_credits',
 'program_id']

In [53]:
student_info

{'user_id': 3,
 'name': None,
 'financial_aid': 1,
 'resident_status': 'In-State',
 'student_profile': 'Full-time',
 'transfer_credits': 0,
 'app_stage': 'schedule created',
 'app_stage_id': 4,
 'first_term': None,
 'program_id': 5,
 'degree_program': 'BS in Business Administration',
 'menu': {'degree': None, 'area_of_study': None, 'program': None},
 'degree_id': 3}